- 정류장 배치가 적절한지
    - 사람들 평균 이동 거리
    - Station간 거리
    - 너무 가깝거나 먼 곳이 있는지 확인
- 정류장 Capacity의 수가 적절한지(나가는 것과 들어오는 것이 적당히 있어야 한다.)
    - 출발 이용 + 끝내는 이용수
    - 현재 용량

In [1]:
import pandas as pd

In [2]:
journeys = pd.read_csv('journeys.csv')
stations = pd.read_csv('stations.csv')

In [3]:
print(journeys.shape)
print(stations.shape)

(1542844, 14)
(773, 5)


In [4]:
journeys.head(2)

,Journey Duration,Journey ID,End Date,End Month,End Year,End Hour,End Minute,End Station ID,Start Date,Start Month,Start Year,Start Hour,Start Minute,Start Station ID
0,2040.0,953,19,9,17,18,0,478,19,9,17,17,26,251
1,1800.0,12581,19,9,17,15,21,122,19,9,17,14,51,550


In [5]:
stations.head(2)

,Station ID,Capacity,Latitude,Longitude,Station Name
0,1,19,51.529163,-0.109970,"River Street , Clerkenwell"
1,2,37,51.499606,-0.197574,"Phillimore Gardens, Kensington"


In [6]:
# 끝 정류장에 대한 정보를 journeys에 추가
data = pd.merge(journeys, stations, left_on = 'End Station ID', right_on = 'Station ID', how = 'left')

# Station ID는 End Station ID와 동일하므로 삭제
data = data.drop(['Station ID'], axis = 1)

# 시작 정류장에 대한 정보도 추가해야되기 때문에 Column값 변경
data.rename(columns={"Capacity":"End Capacity", "Latitude":"End Latitude", "Longitude":"End Longitude", "Station Name":"End Station Name"},inplace = True)

data.head(2)

,Journey Duration,Journey ID,End Date,End Month,End Year,End Hour,End Minute,End Station ID,Start Date,Start Month,Start Year,Start Hour,Start Minute,Start Station ID,End Capacity,End Latitude,End Longitude,End Station Name
0,2040.0,953,19,9,17,18,0,478,19,9,17,17,26,251,21.0,51.521889,-0.046670,"Stepney Green Station, Stepney"
1,1800.0,12581,19,9,17,15,21,122,19,9,17,14,51,550,23.0,51.521113,-0.078869,"Norton Folgate, Liverpool Street"


In [7]:
# 시작 정류장에 대한 정보를 journeys에 추가
data = pd.merge(data, stations, left_on = 'Start Station ID', right_on = 'Station ID', how = 'left')

# Station ID는 Start Station ID와 동일하므로 삭제
data = data.drop(['Station ID'], axis = 1)

# Column값 변경
data.rename(columns={"Capacity":"Start Capacity", "Latitude":"Start Latitude", "Longitude":"Start Longitude", "Station Name":"Start Station Name"},inplace = True)

# 최종 데이터
data.head(2)

,Journey Duration,Journey ID,End Date,End Month,End Year,End Hour,End Minute,End Station ID,Start Date,Start Month,...,Start Minute,Start Station ID,End Capacity,End Latitude,End Longitude,End Station Name,Start Capacity,Start Latitude,Start Longitude,Start Station Name
0,2040.0,953,19,9,17,18,0,478,19,9,...,26,251,21.0,51.521889,-0.046670,"Stepney Green Station, Stepney",34.0,51.518908,-0.079249,"Brushfield Street, Liverpool Street"
1,1800.0,12581,19,9,17,15,21,122,19,9,...,51,550,23.0,51.521113,-0.078869,"Norton Folgate, Liverpool Street",23.0,51.521564,-0.039264,"Harford Street, Mile End"


In [8]:
data.shape

(1542844, 22)

## 1. 정류장 별 나가고 들어오는 자전거 수가 비슷한지?

- 나가는 수와 들어오는 수의 차가 0인지?

In [33]:
# 시작하는 
pd.pivot_table(data, index = 'Start Station ID', values = 'Journey Duration', aggfunc = 'count').head()

,Journey Duration
Start Station ID,
1,1269
2,2055
3,3337
4,1405
5,3335


In [18]:
pd.pivot_table(data, index = 'End Station ID', values = 'Journey Duration', aggfunc = 'count').head()

,Journey Duration
End Station ID,
1,877
2,2379
3,3786
4,1548
5,3596


In [20]:
data_1 = pd.pivot_table(data, index = 'Start Station ID', values = 'Journey Duration', aggfunc = 'count') - pd.pivot_table(data, index = 'End Station ID', values = 'Journey Duration', aggfunc = 'count')
data_1

,Journey Duration
Start Station ID,
1,392
2,-324
3,-449
4,-143
5,-261


In [28]:
data_1.head()

,Journey Duration
Start Station ID,
1,392
2,-324
3,-449
4,-143
5,-261


In [31]:
data_1[(data_1['Journey Duration'] > 1000) | (data_1['Journey Duration'] < -1000)]

,Journey Duration
Start Station ID,
66,-3186
71,-1242
194,-2791
228,-1460
251,-1332
361,2005
374,1299
574,1101
762,-1098


In [34]:
data_1[(data_1['Journey Duration'] > 1000) | (data_1['Journey Duration'] < -1000)].mean()

Journey Duration   -744.888889
dtype: float64

In [37]:
data_1[(data_1['Journey Duration'] > 1000) | (data_1['Journey Duration'] < -1000)]

,Journey Duration
Start Station ID,
66,-3186
71,-1242
194,-2791
228,-1460
251,-1332
361,2005
374,1299
574,1101
762,-1098


In [40]:
data_1[data_1['Journey Duration'] > 0].count()

Journey Duration    437
dtype: int64

In [41]:
data_1[data_1['Journey Duration'] < 0].count()

Journey Duration    337
dtype: int64

In [44]:
data_1.mean()

Journey Duration    0.0
dtype: float64

In [42]:
data_1.sum()

Journey Duration    0
dtype: int64

- 위 데이터에다가 경도, 위도 붙이고 시각화
- count해서 분포 확인(정규분포인지)